![title](https://ucm.es/themes/ucm16/media/img/logo.png)

# Bases de datos NoSQL

### Práctica Redis. Servicios de publicación/subscripción 

#### Parte 2. Subscripción

Lo primero, el nombre:

Nombre

Ahora conectamos a Redis. Es importante que cambieis los datos por los de vuestra cuenta, para no saturar el servidor. Además, los datos deben ser los mismos del publisher, si no no funcionará

In [ ]:
# Ojo, cambiar estos datos por los de vuestro acceso a red en redislabs
redisconexion = "redis-13665.c55.eu-central-1-1.ec2.cloud.redislabs.com"
redispuerto = 13665
passwd = "csVe77ZtQL7sKQocZZHUlnjmSf0WpGxE"

import redis
r = redis.Redis(host=redisconexion, password=passwd, port=redispuerto, charset="utf-8", decode_responses=True, db=0)
r.ping() # debe mostrar True

#### Pregunta 2: 1 punto

Declarar 3 subcriptores, s1, s2 y s3:
    
* s1 estará suscrito solo a 'news-money'

* s2 estará suscrito solo a 'news-selected'

* s3 estará suscrito solo a 'news-selected' y a 'news-other'

In [ ]:
# solución


#### Pregunta 3: 2 puntos

Tras ejecutar el código anterior, ejecutar el publisher, que tardará unos segundos (puede que un minuto). Ahora ya tenemos tanto subcriptores (s1,s2,s3) como mensajes publicados. Para probarlo, escribir una función muestra que imprime la clave 'data' y 'chanel' de los n primeros mensajes  del subscriptor de tipo (type) `message` o `pmessage`.

Nota: suponer que hay n mensajes, no hace falta comprobarlo

In [ ]:
# solución

# s es un subcriptor, n el número de mensajes a mostrar
def muestra(s,n):
   

Para probar el código anterior

In [ ]:
print("s1")
muestra(s1,3)
print("s2")
muestra(s2,3)
print("s3")
muestra(s3,3)

La salida dependerá de las noticias, pero debe seguir el formato:

    s1
    Mensaje  1  Canal  news-money 
    Titular: Maharashtra govt to distribute ₹2,900 cr for drought relief 

    Mensaje  2  Canal  news-money 
    Titular: Fresh milk startup Country Delight raises $7-10 mn: Reports 

    Mensaje  3  Canal  news-money 
    Titular: Deals worth ₹3 lakh cr signed at Global Investors Meet: TN 

    s2
    Mensaje  1  Canal  news-selected 
    Titular: Came back from India trip, amazed at the changes: Mark Mobius 
    ....

#### Pregunta 4: 1 punto

* Crear un nuevo subscriptor s4
* Utilizar la subscripción por patrones para subscribirlo a todos los canales que empiezan por `news-`


Para probar:

In [ ]:
# antes de ejecutar este código ejecutar de nuevo el publisher (la última caja de código)
muestra(s4,4)

#### Pregunta 5: 2 puntos
Lo normal es que no queramos mostrar la información sin más por pantalla, sino que queramos grabar lo publicado en una base de datos. En nuestro caso se pide:

* Crear un servidor de mongo (o conectar a Atlas, como se desee)
* Escribir una función graba(s,db) que grabe cada mensaje en una colección con el mismo nombre que el canal en el que se ha publicado el mensaje. Cada documento solo tendrá una clave `titular` (aparte del _id, que dejaremos que se genera automáticamente)

graba(s,db) grabará en mongo titulares hasta que la función `get_message` devuelva None

**AViso**: el siguiente código borra la base de datos news, asegurarse de no tener nada importante

In [ ]:
# iniciamos mongo
from pymongo import MongoClient
client = MongoClient('mongodb://127.0.0.1:27017/')

## OJO: borramos la base de datos news
client.drop_database('news')
db = client.news

# graba todos los mensajes del publisher s en la base de datos db, con el nombre
# de colecciones que indice 'channel'
def graba(s,db):
  
    
    


Para probarlos nos declaramos un subcriptor

In [ ]:

s5 = r.pubsub() 
s5.subscribe("news-selected")
s5.subscribe("news-money")


Ahora ejecutamos el publisher, y a continuación la llamada a nuestra función, que grabará en mongo en dos colecciones, news-selected, y news-money

In [ ]:
graba(s5,db)

Vamos a ver si se ha insertado bien mirando los primeros documentos de cada coleccion

In [ ]:
def muestra_primeros(db,n):
    for c in db.list_collection_names():
        print("Colección ",c)
        for doc in db[c].find({},{"_id":0}).limit(n):
            print(doc)
       

In [ ]:
muestra_primeros(db,10)

La salida debe tener el aspecto:    
    
    Colección  news-selected
    {'titular': 'US govt agency condemns Pak as food aid being denied to Hindus amid coronavirus'}
    {'titular': 'Coronaviruses found in two bat species in India: ICMR'}
    ....